In [1]:
%load_ext autoreload
%autoreload 2

import ncem
import numpy as np
import seaborn as sns
import tensorflow as tf



from scipy.stats import ttest_rel, ttest_ind

sns.set_palette("colorblind")

# paths
data_path_base ="./input-data/raw-data/"
out_path = "./output-data/Hartmann-2021/"
fn_out_cv = out_path + "/results/"

2023-01-24 01:01:17.129612: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-24 01:01:17.226583: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-01-24 01:01:17.598706: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-01-24 01:01:17.598752: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] 

## If errors occur with CUDA

- If `InternalError: libdevice not found at ./libdevice.10.bc [Op:__inference_one_e_step_2806]`
    - Include in shell $PATH:
        `export XLA_FLAGS=--xla_gpu_cuda_data_dir=/usr/lib/cuda`


In [2]:
# - if `"Attempting to perform BLAS operation using StreamExecutor without BLAS support`
    # - try setting a dedicate amount of GPU vram:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        tf.config.experimental.set_virtual_device_configuration(gpus[0],[tf.config.experimental.VirtualDeviceConfiguration(memory_limit=500)]) # 1500 [MB]
    except RuntimeError as e:
        print('ERROR')

2023-01-24 01:01:19.004372: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-24 01:01:19.012630: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-24 01:01:19.012744: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero


# Dataset specific inputs

In [3]:
data_set = 'hartmann'
data_path = data_path_base + '/Hartmann-2021/'
log_transform = False
use_domain = True
scale_node_size=False
merge_node_types_predefined = True
covar_selection = []
output_layer='gaussian'


# Manual inputs

In [4]:
model_class = 'cvae'
optimizer = 'adam'
domain_type = 'patient'

learning_rate = 0.05
l1 = 0.
l2 = 0.

batch_size = 58
radius = 10
n_eval_nodes = 10

gs_id = f"tutorial_{model_class}_{radius}_{data_set}_{domain_type}"



# Model and training


In [5]:
ncv = 3
epochs = 2000 if "tutorial" not in gs_id else 10 
epochs_warmup = 0
max_steps_per_epoch = 20
patience = 100
lr_schedule_min_lr = 1e-10
lr_schedule_factor = 0.5
lr_schedule_patience = 50
val_bs = 16
max_val_steps_per_epoch = 10
shuffle_buffer_size = None
decoder_epochs=1000
decoder_patience=20

feature_space_id = "standard"
cond_feature_space_id = "type"

use_covar_node_label = False
use_covar_node_position = False
use_covar_graph_covar = False

In [6]:
trainer = ncem.train.TrainModelCVAE()
trainer.init_estim(log_transform=log_transform)

trainer.estimator.get_data(
    data_origin=data_set,
    data_path=data_path,
    radius=radius,
    graph_covar_selection=covar_selection,
    node_label_space_id=cond_feature_space_id,
    node_feature_space_id=feature_space_id,
    use_covar_node_position=use_covar_node_position,
    use_covar_node_label=use_covar_node_label,
    use_covar_graph_covar=use_covar_graph_covar,
    domain_type=domain_type,
)

/home/ifichtner/mambaforge/envs/squidpyTF/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loading data from raw files
registering celldata


/home/ifichtner/mambaforge/envs/squidpyTF/lib/python3.8/site-packages/ncem/data.py:2389: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  celldata = AnnData(
/home/ifichtner/mambaforge/envs/squidpyTF/lib/python3.8/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


collecting image-wise celldata
adding graph-level covariates


100%|██████████| 58/58 [00:00<00:00, 59.08it/s]

Loaded 58 images with complete data from 4 patients over 63747 cells with 36 cell features and 8 distinct celltypes.
Mean of mean node degree per images across images: 0.000688


In [7]:
trainer.estimator.split_data_node(
    validation_split=0.1,
    test_split=0.1,
    seed=0
)

Using split method: node. 
 Train-test-validation split is based on total number of nodes per patients over all images.

Excluded 0 cells with the following unannotated cell type: [None] 

Whole dataset: 63747 cells out of 58 images from 4 patients.
Test dataset: 6376 cells out of 58 images from 4 patients.
Training dataset: 51930 cells out of 58 images from 4 patients.
Validation dataset: 5738 cells out of 58 images from 4 patients. 



In [8]:
trainer.estimator.init_model(
    optimizer=optimizer,
    learning_rate=learning_rate,
    n_eval_nodes_per_graph=n_eval_nodes,

    l1_coef=l1,
    l2_coef=l2,
    use_domain=use_domain,
    use_batch_norm=False,
    scale_node_size=scale_node_size,
    output_layer=output_layer,

    latent_dim=10,
    dropout_rate=0.1,
    intermediate_dim_enc=128,
    intermediate_dim_dec=128,
    depth_enc=1,
    depth_dec=1,
)
trainer.estimator.model.training_model.summary()

2023-01-24 01:01:23.752578: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-24 01:01:23.753386: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-24 01:01:23.753549: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-24 01:01:23.753625: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there

Model: "cvae"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_da_group (InputLayer)    [(None, 4)]          0           []                               
                                                                                                  
 tf.cast (TFOpLambda)           (None, 4)            0           ['input_da_group[0][0]']         
                                                                                                  
 tf.expand_dims (TFOpLambda)    (None, 1, 4)         0           ['tf.cast[0][0]']                
                                                                                                  
 categorical_predictor (InputLa  [(None, 10, 0)]     0           []                               
 yer)                                                                                          

In [9]:
trainer.estimator.train(
    epochs=epochs,
    epochs_warmup=epochs_warmup, # Integer number of times to iterate over the training data arrays in warm up (without early stopping)
    max_steps_per_epoch=max_steps_per_epoch,
    batch_size=batch_size,
    validation_batch_size=val_bs,
    max_validation_steps=max_val_steps_per_epoch,
    patience=patience, # Number of epochs with no improvement.
    lr_schedule_min_lr=lr_schedule_min_lr,
    lr_schedule_factor=lr_schedule_factor,
    lr_schedule_patience=lr_schedule_patience,
    monitor_partition="val",
    monitor_metric="loss",
    shuffle_buffer_size=shuffle_buffer_size,
    early_stopping=True,
    reduce_lr_plateau=True,
    decoder_epochs=decoder_epochs,
    decoder_patience=decoder_patience
)

Epoch 1/10
Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089
2023-01-24 01:01:26.493192: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:630] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-01-24 01:01:26.497497: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x561477ab8260 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-01-24 01:01:26.497516: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): NVIDIA GeForce RTX 3050 Laptop GPU, Compute Capability 8.6
2023-01-24 01:01:26.502156: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-01-24 01:01:26.600346: I tensorflow/compiler/jit/xla_compilation_ca

1/1 - 3s - loss: 41.4637 - reconstruction_loss: 41.4549 - bottleneck_loss: 0.0088 - reconstruction_custom_mae: 0.5458 - reconstruction_custom_mean_sd: 1.0000 - reconstruction_custom_mse: 0.4652 - reconstruction_custom_mse_scaled: 0.4652 - reconstruction_gaussian_reconstruction_loss: 41.4549 - reconstruction_r_squared: -4.6231e+00 - reconstruction_r_squared_linreg: 8.6166e-04 - bottleneck_custom_kl: 0.8810 - elbo: 41.5430 - val_loss: 77.2826 - val_reconstruction_loss: 76.8920 - val_bottleneck_loss: 0.3906 - val_reconstruction_custom_mae: 1.3017 - val_reconstruction_custom_mean_sd: 0.9568 - val_reconstruction_custom_mse: 2.3189 - val_reconstruction_custom_mse_scaled: 2.5228 - val_reconstruction_gaussian_reconstruction_loss: 76.8920 - val_reconstruction_r_squared: -2.8670e+01 - val_reconstruction_r_squared_linreg: 0.1542 - val_bottleneck_custom_kl: 39.0605 - val_elbo: 80.7981 - lr: 0.0500 - 3s/epoch - 3s/step
Epoch 2/10
1/1 - 0s - loss: 81.0108 - reconstruction_loss: 80.5984 - bottleneck_

In [10]:
evaluation_test = trainer.estimator.evaluate_any(
    img_keys=trainer.estimator.img_keys_test,
    node_idx=trainer.estimator.nodes_idx_test
)

In [11]:
split_per_node_type, evaluation_per_node_type = trainer.estimator.evaluate_per_node_type()

Evaluation for CD11c Myeloid with 1199 cells
{'loss': 50.795108795166016, 'reconstruction_loss': 49.46112060546875, 'bottleneck_loss': 1.3339920043945312, 'reconstruction_custom_mae': 0.618635356426239, 'reconstruction_custom_mean_sd': 1.1805644035339355, 'reconstruction_custom_mse': 0.6972304582595825, 'reconstruction_custom_mse_scaled': 0.5860023498535156, 'reconstruction_gaussian_reconstruction_loss': 49.46112060546875, 'reconstruction_r_squared': -7.505324840545654, 'reconstruction_r_squared_linreg': 0.006172799970954657, 'bottleneck_custom_kl': 133.39918518066406, 'elbo': 62.801029205322266}
Evaluation for CD4 T cells with 14983 cells
{'loss': 49.7149658203125, 'reconstruction_loss': 48.35279083251953, 'bottleneck_loss': 1.3621814250946045, 'reconstruction_custom_mae': 0.5964818596839905, 'reconstruction_custom_mean_sd': 1.1805492639541626, 'reconstruction_custom_mse': 0.6357350945472717, 'reconstruction_custom_mse_scaled': 0.5244260430335999, 'reconstruction_gaussian_reconstructi

In [12]:
evaluation_per_node_type['Fibroblast']

{'loss': 48.6131477355957,
 'reconstruction_loss': 47.417274475097656,
 'bottleneck_loss': 1.1958887577056885,
 'reconstruction_custom_mae': 0.5758668184280396,
 'reconstruction_custom_mean_sd': 1.180565357208252,
 'reconstruction_custom_mse': 0.5665685534477234,
 'reconstruction_custom_mse_scaled': 0.4724542498588562,
 'reconstruction_gaussian_reconstruction_loss': 47.41725540161133,
 'reconstruction_r_squared': -6.493439197540283,
 'reconstruction_r_squared_linreg': 0.03642991557717323,
 'bottleneck_custom_kl': 119.58879852294922,
 'elbo': 59.37611389160156}